In [5]:
#pip install opencv-python mediapipe pyautogui


In [1]:
import cv2
import mediapipe as mp
import pyautogui


In [2]:
cap = cv2.VideoCapture(0)
hand_detector = mp.solutions.hands.Hands(max_num_hands=1)
draw = mp.solutions.drawing_utils

screen_width, screen_height = pyautogui.size()


In [3]:
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)  # mirror the image
    frame_height, frame_width, _ = frame.shape

    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = hand_detector.process(rgb_frame)

    hands = output.multi_hand_landmarks
    if hands:
        for hand in hands:
            draw.draw_landmarks(frame, hand, mp.solutions.hands.HAND_CONNECTIONS)

            landmarks = hand.landmark
            index_finger_tip = landmarks[8]  # index finger tipq
            thumb_tip = landmarks[4]        # thumb tip

            # Convert coordinates
            x = int(index_finger_tip.x * frame_width)
            y = int(index_finger_tip.y * frame_height)

            screen_x = screen_width / frame_width * x
            screen_y = screen_height / frame_height * y

            pyautogui.moveTo(screen_x, screen_y)

            # Pinch detection (for click)
            thumb_x = int(thumb_tip.x * frame_width)
            thumb_y = int(thumb_tip.y * frame_height)
            distance = ((x - thumb_x) ** 2 + (y - thumb_y) ** 2) ** 0.5

            if distance < 40:
                pyautogui.click()
                cv2.circle(frame, (x, y), 15, (0, 255, 0), cv2.FILLED)

    cv2.imshow("Virtual Mouse", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
